Diseño de Lenguajes de Programación
Jennifer Michelle Toxcon Ordoñez | 21276<br>

## Algoritmo de Shunting Yard


In [1]:
def insert_concatenation(expression):
    result = []
    operators = "+|*()"
    for i in range(len(expression)):
        char = expression[i]
        result.append(char)

        if i + 1 < len(expression):
            lookahead = expression[i + 1]

            if char.isalnum() and lookahead not in operators and lookahead != '.':
                result.append('.')
            elif char == '*' and lookahead.isalnum():
                result.append('.')
            elif char == ')' and lookahead.isalnum():
                result.append('.')
            elif char.isalnum() and lookahead == '(':
                result.append('.')
            elif char == ')' and lookahead == '(':
                result.append('.')


    return ''.join(result)

def shunting_yard(expression):
     nueva_expresion = reemplazar_signo(expression)
     precedence = {'+': 1, '|': 1, '*': 3, '.': 2} # Precedence of operators

     output_queue = [] # Output queue (postfix expression)
     operator_stack = [] # Operator stack
     i = 0

     expression = insert_concatenation(nueva_expresion)
     print("expression", expression)

     while i < len(expression):
         token = expression[i]
         if token.isalnum() or token == 'ε':
             output_queue.append(token)
         elif token in "+|*.":
             while (operator_stack and operator_stack[-1] != '(' and
                    precedence[token] <= precedence.get(operator_stack[-1], 0)):
                 output_queue.append(operator_stack.pop())
             operator_stack.append(token)
         elif token == '(':
             operator_stack.append(token)
         elif token == ')':
             while operator_stack and operator_stack[-1] != '(':
                 output_queue.append(operator_stack.pop())
             operator_stack.pop()
         elif token == '.':
             while operator_stack and operator_stack[-1] != '(':
                 output_queue.append(operator_stack.pop())
             if operator_stack[-1] == '(':
                 operator_stack.pop()
         i += 1

     while operator_stack:
         output_queue.append(operator_stack.pop())

     if not output_queue:
         return 'ε'
     else:
         final = ''.join(output_queue)
         if final.endswith('+'):
            print("final", final)
            final = final[:-1]
         return final


def reemplazar_signo(expresion):
    nueva_expresion = ''
    i = 0

    while i < len(expresion):
        if expresion[i] == '?' and i + 1 < len(expresion) and expresion[i + 1] == '|':
            nueva_expresion += '|ε'
            i += 2
        elif expresion[i] == '?':
            if i - 1 >= 0:
                nueva_expresion = nueva_expresion[:-1] + '(' + nueva_expresion[-1] + '|ε)'
            else:
                nueva_expresion += '|ε'
            i += 1
        elif expresion[i] == '+':
            if i - 1 >= 0:
                nueva_expresion += expresion[i - 1] + expresion[i] + '*'
            else:
                nueva_expresion += expresion[i] + '*'
            i += 1
        else:
            nueva_expresion +=  expresion[i]
            i += 1

    nueva_expresion = nueva_expresion.replace('+', "")
    return nueva_expresion

# Example of use
infix_exp = input("Ingrese la expresion regular: ")

# Input example: (a|b)*.a.b
postfix_expression = shunting_yard(infix_exp)

print("Postfix expression:", postfix_expression)

KeyboardInterrupt: Interrupted by user

## Algoritmo de Thompson

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from typing import *
import graphviz
import sys

class Node:
    def __init__(self, value: str, left: 'Node' or None = None, right: 'Node' or None = None, id_: int or None = None) \
            -> None:
        self.value: str = value
        self.left: 'Node' or None = left
        self.right: 'Node' or None = right
        self.id_: int or None = id_
        self.is_nullable: bool = False
        self.first_pos: Set[int] = set()
        self.last_pos: Set[int] = set()
        self.isLeft: bool = False
        self.follow_pos: Set[str] = set()

    def getId(self) -> str:
        return str(id(self))

class State:
    def __init__(self, value: str) -> None:
        self.value: str = value
        self.transitions: Dict[str, Set['State']] = {}
        self.isFinalState: bool = False

    def add_transition(self, value: str, state: 'State') -> None:
        if value in self.transitions:
            self.transitions[value].add(state)
        else:

            self.transitions[value] = {state}

    def getId(self) -> str:
        return str(id(self))

    def getStates(self, transition_value) -> Set['State']:
        return self.transitions[transition_value] if transition_value in self.transitions else set()

    def __eq__(self, other):
        if isinstance(other, State):
            return (self.value, id(self)) == (other.value, id(self))
        return False

    def __hash__(self):
        return hash((self.value, id(self)))

    def getEpsilonClean(self):
        states: Set['State'] = self.getStates('ε')
        not_explorer: Set['State'] = set()

        for state in states:
            not_explorer = not_explorer.union(state.getStates('ε'))

        not_explorer = not_explorer - states

        while len(not_explorer) > 0:
            states = states.union(not_explorer)
            copy = not_explorer.copy()
            not_explorer = set()
            for state in copy:
                not_explorer = not_explorer.union(state.getStates('ε'))
            not_explorer = not_explorer - states

        return states.union({self})

    def delState(self, state: 'State'):
        for transition in self.transitions:
            if state in self.transitions[transition]:
                self.transitions[transition].remove(state)

def make_tree(expression: str) -> Node:
    stack = []
    id_ = 1
    for char in expression:
        if char == '.':
            if len(stack) < 2:
              print("Error: Insufficient operands for concatenation.")
              sys.exit(1)
            right = stack.pop()
            left = stack.pop()
            stack.append(Node(char, left, right))
            stack[-1].is_nullable = left.is_nullable and right.is_nullable
            if left.is_nullable:
                stack[-1].first_pos = left.first_pos.union(right.first_pos)
            else:
                stack[-1].first_pos = left.first_pos
            if right.is_nullable:
                stack[-1].last_pos = left.last_pos.union(right.last_pos)
            else:
                stack[-1].last_pos = right.last_pos
        elif char == '|':
            right = stack.pop()
            left = stack.pop()
            stack.append(Node(char, left, right))
            stack[-1].is_nullable = left.is_nullable or right.is_nullable
            stack[-1].first_pos = left.first_pos.union(right.first_pos)
            stack[-1].last_pos = left.last_pos.union(right.last_pos)
        elif char == '*':
            left = stack.pop()
            stack.append(Node(char, left))
            stack[-1].is_nullable = True
            stack[-1].first_pos = left.first_pos
            stack[-1].last_pos = left.last_pos
        else:
            stack.append(Node(char, id_=id_))
            stack[-1].is_nullable = char == 'ε'
            stack[-1].first_pos.add(id_)
            stack[-1].last_pos.add(id_)
            id_ += 1
    return stack.pop()

def afn_for_simulation(regex,index):
    postfix = shunting_yard(regex)
    regex = reemplazar_signo(regex)
    #postfix = 'c.ab|*'
    stack = []
    accept_state = []
    state_count = 0
    previous_symbol = None
    simulationAFN = nx.DiGraph()
    simulationAFN.add_node(state_count)
    start_state = state_count

    epsilon_state = state_count + 1
    simulationAFN.add_node(epsilon_state)
    simulationAFN.add_edge(state_count, epsilon_state, label='ε')
    state_count += 1

    # Mantén un seguimiento de los estados en cada nivel de alternancia
    alt_states = [set([1])]
    boolean = False
    cont = 0

    count_dots = 0

    for symbol in postfix:
        if symbol == '.':
            count_dots += 1

    for symbol in postfix:

        if symbol.isalnum():
            state_count += 1
            cont +=1
            simulationAFN.add_node(state_count)
            simulationAFN.add_edge(state_count - 1, state_count, label=symbol)
            #print("cont",cont)
            stack.append(state_count)
        elif symbol == '*':
            state = stack.pop()
            char = regex[state - 2]

            if char == '(':
              char = regex[state - 1]
            elif char == '*':
              char = regex[state - 7]
            elif char == 'ε':
              char = regex[state-1]

            if previous_symbol != '|' and previous_symbol != '.':
               simulationAFN.add_edge(state, state, label=previous_symbol)
            elif previous_symbol == '.':
              simulationAFN.add_edge(state, state-(count_dots + 1), label='ε')

            #print("symbol",symbol)
            #print("Anterior", previous_symbol)
            #afn.add_edge(state, state, label=char)
            #afn.add_edge(state_count, state, label='ε')

            simulationAFN.add_edge(state_count, state_count + 1, label='ε')
            if previous_symbol == '|' and cont <= 2:
              simulationAFN.add_edge(epsilon_state, state_count + 1, label='ε')
            #else:
            #  afn.add_edge(epsilon_state, state_count + 1, label='ε')

            epsilon_state = state_count + 2

            state_count += 1
            simulationAFN.add_node(state_count)
            stack.append(state_count)

            #accept_state += [state]
            #accept_state += [state - 1]
            if count_dots >=1 and state <=5:
              simulationAFN.add_edge(state-(count_dots+1), state_count , label='ε')
              #accept_state += [state - (count_dots-1)]
            elif count_dots >=1 and state ==6:
              i = state - 5
              simulationAFN.add_edge(state-(count_dots+2+i), state_count , label='ε')
              #accept_state += [state - (count_dots-1)]
            elif count_dots >=1 and state >6:
              i = 7 - state
              simulationAFN.add_edge(state-(count_dots+2+i), state_count , label='ε')
              #accept_state += [state - (count_dots-1)]

        elif symbol == '|':
              state2 = stack.pop()
              if len(stack) == 0:
                char1 = 'ε'
                state1 = 3
              else:
                state1 = stack.pop()
                char1 = regex[count_dots-state1]
                #char1 = regex[state1-3]
                #print("char1: ",char1)
                if char1 == ')':
                  char1 = regex[state1 - 1]
                  if char1 == '*':
                    char1 = 'ε'
                elif char1 == '*':
                  char1 = regex[state1 - 1]
                  boolean = True


              #print("state1: ",state1)


              #if char1 == '(':
                #char1 = 'ε'


              char2 = regex[state2 - 3]
              if char2 == '(':
                char2 = regex[state2 ]

              if char2 == ')':
                char2 = regex[state2-1]

              if char2 == 'b' and char1 == 'b':
                char2 = regex[state2-2]
              elif char2 == '*':
                char2 == 'ε'
              elif char2 == '|':
                char2 = regex[state2-4]
              elif count_dots>0 and char1 == 'ε':
                char2 = regex[state2-count_dots]

              state_count += 1
              #afn.add_node(state_count)

              if index + 1 < len(postfix):
                sig = postfix[index+1]
              else:
                sig = None

              if cont >= 3:
                simulationAFN.add_edge(epsilon_state+(cont-2), state1, label=char1)
                simulationAFN.add_edge(epsilon_state+(cont-2), state2, label=char2)

              elif cont <3 and epsilon_state<4:
                simulationAFN.add_edge(epsilon_state, state1, label=char1)
                simulationAFN.add_edge(epsilon_state, state2, label=char2)

              elif cont <3 and epsilon_state>=4:
                simulationAFN.add_edge(state1-2, state1, label=char1)
                #afn.add_edge(epsilon_state+(cont-2), state2, label=char2)


      #        if boolean is True:
      #          afn.add_edge(state1, state1, label=char1)

              simulationAFN.add_edge(state1, state_count + 1, label='ε')
              simulationAFN.add_edge(state2, state_count + 1, label='ε')

              if sig == '*':
                  #Transiciones Reflexivas
                  simulationAFN.add_edge(state_count + 1, state1 - 1, label='ε')
                  #afn.add_edge(state2, state2, label=char2)


              state_count += 1
              # Verifica si la arista existe antes de intentar eliminarla
              if simulationAFN.has_edge(state1, state2):
                  simulationAFN.remove_edge(state1, state2)
              #afn.remove_edge(state1,state2)
              #afn.add_node(state_count)
              stack.append(state_count)
        elif symbol == '.':

            state2 = stack.pop()
            if len(stack) == 0:
              char1 = 'ε'
              state1 = 3
            else:
              state1 = stack.pop()

            #afn.add_edge(state1, state2, label='ε')
            stack.append(state2)
            #accept_state += [state1]
        index += 1
        previous_symbol = symbol


    final_state = state_count +1
    accept_state += [final_state]
    simulationAFN.add_node(final_state)
    simulationAFN.add_edge(state_count, final_state, label='ε')

    simulationAFN.graph['start'] = start_state
    simulationAFN.graph['accept'] = accept_state

    return simulationAFN, accept_state


def regex_to_afn(tree: Node, initState: State = State(str(0))):
    noInit = int(initState.value)

    if tree.value == '|':
        left, noInit, leftFinal_state = regex_to_afn(tree.left, State(str(noInit+1)))
        right, noInit, rightFinal_state = regex_to_afn(tree.right, State(str(noInit+1)))
        initState.add_transition('ε', left)
        initState.add_transition('ε', right)
        finalState = State(str(noInit + 1))
        leftFinal_state.add_transition('ε', finalState)
        rightFinal_state.add_transition('ε', finalState)

    elif tree.value == '.':
        left, noInit, leftFinal_state = regex_to_afn(tree.left, initState)
        right, noInit, rightFinal_state = regex_to_afn(tree.right, leftFinal_state)
        finalState = rightFinal_state

    elif tree.value == '*':
        left, noInit, leftFinal_state = regex_to_afn(tree.left, State(str(noInit+1)))
        initState.add_transition('ε', left)
        finalState = State(str(noInit+1))
        leftFinal_state.add_transition('ε', finalState)
        initState.add_transition('ε', finalState)
        leftFinal_state.add_transition('ε', left)

    else:
        finalState = State(str(noInit + 1))
        initState.add_transition(tree.value, finalState)

    return initState, int(finalState.value), finalState



def compute_epsilon_closure(afn, state):
    epsilon_closure = set()
    stack = [state]

    while stack:
        current_state = stack.pop()
        epsilon_closure.add(current_state)

        for successor, edge_data in afn.adj[current_state].items():
            label = edge_data.get('label', None)
            if label == 'ε' and successor not in epsilon_closure:
                stack.append(successor)

    return epsilon_closure



def move(afn, state, symbol):
    target_states = set()

    for successor, edge_data in afn.adj[state].items():
        label = edge_data.get('label', None)
        if label == symbol:
            target_states.add(successor)

    return target_states


def get_alphabet(afn):
    alphabet = set()

    for _, _, label in afn.edges(data='label'):
        if label != 'ε':
            alphabet.add(label)

    return alphabet

def epsilon_closure(afn, states):
    closure = set(states)
    stack = list(states)
    while stack:
        state = stack.pop()
        for successor, attributes in afn[state].items():
            label = attributes['label']
            if label == 'ε' and successor not in closure:
                closure.add(successor)
                stack.append(successor)
            elif label == '*':
                closure.add(successor)
                for epsilon_successor in epsilon_closure(afn, {successor}):
                    if epsilon_successor not in closure:
                        closure.add(epsilon_successor)
                        stack.append(epsilon_successor)
    return closure

def check_membership(afn, s):
    current_states = epsilon_closure(afn, {afn.graph['start']})
    for symbol in s:
        next_states = set()

        for state in current_states:
            for successor, attributes in afn[state].items():
                if attributes['label'] == symbol:
                    next_states |= epsilon_closure(afn, {successor})
                    print("Estado actual: ",state)
                    print("Posibles caminos: ",afn[state])
                    print("Lee simbolo: ",symbol)
            if symbol != '*':
                current_states = next_states
    return any (state in afn.graph['accept'] for state in current_states)


if __name__ == "__main__":
    regex = input("Enter a regular expression: ")
    postfix = shunting_yard(regex)
    #print("postfix in thomp: ", postfix)
    tree = make_tree(postfix)
    afnInit, _, finalS = regex_to_afn(tree)


    w = input("Enter a string to check: ")

    afn, accept_state = afn_for_simulation(regex,0)

    # Visualization
    dot: 'graphviz.graphs.Digraph' = graphviz.Digraph(comment='AFN')
    dot.attr(rankdir='LR')
    setStates = set()

    dot.attr(label='AFN')

    def draw_state(state: 'State'):
        setStates.add(state.getId())
        dot.node(state.getId(), label=state.value, shape='doublecircle' if state.isFinalState else 'circle')
        for transition in state.transitions:
            for destiny in state.transitions[transition]:
                if destiny.getId() not in setStates:
                    draw_state(destiny)
                dot.edge(state.getId(), destiny.getId(), label=transition)

    draw_state(afnInit)

    dot.render('AFN'+'.gv', view=True, directory='./AFN')


    # Obtener el conjunto de símbolos
    simbolos = set(label for _, _, label in afn.edges(data='label'))

    # Obtener el conjunto de estados iniciales
    estados_iniciales = {nodo for nodo in afn.nodes() if len(list(afn.predecessors(nodo))) == 0}

    estados_aceptacion = {nodo for nodo in afn.nodes() if len(list(afn.successors(nodo))) == 0}


    # Nombre del archivo de salida
    nombre_archivo = "descripcion_afn.txt"


    # Crear y escribir en el archivo de texto
    with open(nombre_archivo, "w") as archivo:
        archivo.write("ESTADOS = " + str(afn.nodes) + "\n")
        archivo.write("SIMBOLOS = " + str(simbolos) + "\n")
        archivo.write("INICIO = " + str(estados_iniciales) + "\n")
        archivo.write("ACEPTACION =" + str(accept_state) + "\n")
        archivo.write("TRANSICIONES =" + str(afn.edges(data='label')))

    print(f"\nSe ha creado el archivo '{nombre_archivo}' con la descripción del AFN.")
    #SIMULACION DEL AFN
    result = check_membership(afn, w)
    print(result)
    if result:
        print(f"'{w}' pertenece al lenguaje L({regex})")
    else:
        print(f"'{w}' no pertenece al lenguaje L({regex})")

: 

## Construccion de Subconjuntos

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def afn_to_afd(initState: State, alpha: str):
    actualSt: Set[State] = initState.getEpsilonClean()
    stDict: Dict[str, Set[State]] = {'q0': actualSt}
    states: Dict[str, State] = {'q0': State('q0')}
    evaluate = 0
    alphabet = set(list(alpha))
    toEvaluate: List[str] = ['q0']
    while len(toEvaluate) > 0:
        actualSt = stDict[toEvaluate[0]]
        actualState: State = states[toEvaluate[0]]
        if len(toEvaluate) > 1:
            toEvaluate = toEvaluate[1:]
        else:
            toEvaluate.clear()

        preEpsilonStates: Dict[str, Set[State]] = dict()
        epsilonStates: Dict[str, Set[State]] = dict()
        for state in actualSt:
            for letter in alphabet:
                if letter in state.transitions:
                    if letter in preEpsilonStates:
                        preEpsilonStates[letter] = preEpsilonStates[letter].union(state.getStates(letter))
                    else:
                        preEpsilonStates[letter] = state.getStates(letter)

        for letter in preEpsilonStates:
            epsilonStates[letter] = set()
            for state in preEpsilonStates[letter]:
                epsilonStates[letter] = epsilonStates[letter].union(state.getEpsilonClean())

        for letter in epsilonStates:
            if epsilonStates[letter] not in stDict.values():
                evaluate += 1
                index = 'q' + str(evaluate)
                states[index] = State(index)
                stDict[index] = epsilonStates[letter]
                toEvaluate.append(index)
                actualState.add_transition(letter, states[index])
            else:
                for i in stDict:
                    if epsilonStates[letter] == stDict[i]:
                        index = i
                        actualState.add_transition(letter, states[index])

    for index in stDict:
        for statesSt in stDict[index]:
            if statesSt.isFinalState:
                states[index].isFinalState = True

    return states

def Simulation_AFD(afn):
    start_state = afn.graph['start']
    epsilon_closure = compute_epsilon_closure(afn, start_state)

    dfa = nx.DiGraph()
    dfa_start_state = tuple(sorted(epsilon_closure))
    dfa.add_node(dfa_start_state)

    unmarked_states = [dfa_start_state]

    while unmarked_states:
        current_dfa_state = unmarked_states.pop()

        for symbol in get_alphabet(afn):
            target_states = set()
            for afn_state in current_dfa_state:
                target_states.update(move(afn, afn_state, symbol))

            epsilon_closure_target = set()
            for target_state in target_states:
                epsilon_closure_target.update(compute_epsilon_closure(afn, target_state))

            dfa_target_state = tuple(sorted(epsilon_closure_target))

            if dfa_target_state not in dfa:
                unmarked_states.append(dfa_target_state)
                dfa.add_node(dfa_target_state)

            dfa.add_edge(current_dfa_state, dfa_target_state, label=symbol)

    dfa.graph['start'] = dfa_start_state

    # Determine accept states in DFA
    dfa_accept_states = [state for state in dfa.nodes if any(afn_state in afn.graph['accept'] for afn_state in state)]
    print("dfa_accept_state:", dfa_accept_states)
    dfa.graph['accept'] = dfa_accept_states
    #print("nodes:",dfa.edges)
    return dfa


if __name__ == "__main__":

    w = input("Enter a string to check: ")
    alphabet = ''.join(caracter for caracter in regex if caracter.isalnum() and caracter != 'ε')
    #Convierte el afd a afn
    VizAFD = afn_to_afd(afnInit,alphabet)
    print(alphabet)



    afd = Simulation_AFD(afn)

    # Elimina el estado final vacío '()' y sus aristas del AFD
    if ((), ()) in afd.nodes:
        afd.remove_node(((), ()))

        # Asegúrate de también eliminar cualquier arista que apunte a este nodo
        for source, target in list(afd.edges):
            if target == ((), ()):
                afd.remove_edge(source, target)

    # Filtrar las aristas que no tienen tuplas vacías en ambos extremos


    filtered_edges = [(source, target, label) for source, target, label in afd.edges(data='label') if source != () and target != ()]

    # Filtrar los nodos que no son tuplas vacías
    filtered_nodes = [node for node in afd.nodes if node != ()]



    simbolos = set(label for _, _, label in afd.edges(data='label'))
    # Obtener el conjunto de estados iniciales
    estados_iniciales = {nodo for nodo in filtered_nodes if len(list(afd.predecessors(nodo))) == 0}

    estados_aceptacion = set()
    for nodo in filtered_nodes:
      aceptacion = True
      for succ in afd.successors(nodo):
          if succ not in filtered_nodes or succ == ():
              aceptacion = False
              break
      if aceptacion:
          estados_aceptacion.add(nodo)

    print("estados_aceptacion: ",estados_aceptacion)
    # Visualization
    dot: 'graphviz.graphs.Digraph' = graphviz.Digraph(comment='AFD')
    dot.attr(rankdir='LR')
    setStates = set()

    dot.attr(label='AFD')

    def draw_state(state: 'State'):
        setStates.add(state.getId())
        dot.node(state.getId(), label=state.value, shape='doublecircle' if state.isFinalState else 'circle')
        for transition in state.transitions:
            for destiny in state.transitions[transition]:
                if destiny.getId() not in setStates:
                    draw_state(destiny)
                dot.edge(state.getId(), destiny.getId(), label=transition)

    draw_state(VizAFD['q0'])

    dot.render('AFD'+'.gv', view=True, directory='./AFD')

    # Nombre del archivo de salida
    nombre_archivo = "descripcion_afd.txt"


    # Crear y escribir en el archivo de texto
    with open(nombre_archivo, "w") as archivo:
        archivo.write("ESTADOS = " + str(filtered_nodes) + "\n")
        archivo.write("SIMBOLOS = " + str(simbolos) + "\n")
        archivo.write("INICIO = " + str(estados_iniciales) + "\n")
        archivo.write("ACEPTACION =" + str(estados_aceptacion) + "\n")
        archivo.write("TRANSICIONES =" + str(filtered_edges))

    #SIMULACION DEL AFD
    #result = check_membership(afd, w)

    if result:
        print(f"'{w}' belongs to L({regex})")
    else:
        print(f"'{w}' does not belong to L({regex})")


: 

## Algoritmo de Hopcroft


In [ ]:
from collections import deque
import networkx as nx
import matplotlib.pyplot as plt


def hopcroft_minimization(states2: Dict[str, State], alpha: str):
    initSt = states2['q0']
    initState = tuple([states2[x] for x in states2 if not states2[x].isFinalState])
    finalState = tuple([states2[x] for x in states2 if states2[x].isFinalState])
    minimized_states: Dict[str, Tuple[State, ...]] = {'Q0': initState, 'Q1': finalState}

    not_toDo = True

    while not_toDo:
        evaluated = 1
        new_minimized_states: Dict[str, Tuple[State, ...]] = dict()
        for subStates in minimized_states:
            transitionsDict: Dict[str, Set[State]] = dict()
            for minState in minimized_states[subStates]:
                transitions: List[Tuple[str, str]] = []
                for letter in alpha:
                    if len(minState.getStates(letter)) <= 0:
                        pass
                    else:
                        newState = minState.getStates(letter).copy().pop()
                        for subStates2 in minimized_states:
                            if newState in minimized_states[subStates2]:
                                transition: Tuple[str, str] = (subStates2, letter)
                                transitions.append(transition)

                tupleTransitions: str = str(transitions)
                if tupleTransitions in transitionsDict:
                    transitionsDict[tupleTransitions].add(minState)
                else:
                    transitionsDict[tupleTransitions] = {minState}

            for transition_ in transitionsDict:
                if initSt in transitionsDict[transition_]:
                    new_minimized_states['Q' + str(0)] = tuple(transitionsDict[transition_])
                    continue
                new_minimized_states['Q' + str(evaluated)] = tuple(transitionsDict[transition_])
                evaluated += 1

        if new_minimized_states == minimized_states:
            not_toDo = False
        else:
            minimized_states = new_minimized_states

    newMin_States: Dict[str, State] = dict()
    initial = ''
    for subStates in minimized_states:
        index = subStates

        if states2['q0'] in minimized_states[subStates]:
            initial = index
        newMin_States[index] = State(index)

        for minState in minimized_states[subStates]:
            if minState.isFinalState:
                newMin_States[index].isFinalState = True
                break

    for subStates in minimized_states:
        index = subStates
        tryState: State = tuple(minimized_states[subStates])[0]
        newMinState: State = newMin_States[index]

        for letter in alpha:
            if len(tryState.getStates(letter)) <= 0:
                continue

            tranState = tryState.getStates(letter).copy().pop()
            for subStates2 in minimized_states:
                if tranState in minimized_states[subStates2]:
                    newMinState.add_transition(letter, newMin_States[subStates2])
                    break

    newMin_States[initial].value = 'Q0'

    return newMin_States[initial]

def Simulation_AFDMinimal(dfa):

    partitions = [dfa.graph['accept'], list(set(dfa.nodes) - set(dfa.graph['accept']))]
    worklist = deque([dfa.graph['accept']])

    while worklist:
        partition = worklist.popleft()

        for symbol in get_alphabet(dfa):
            divided_partitions = []
            for p in partitions:
                divided = set()
                for state in p:
                    successors = set(dfa.successors(state))
                    if symbol in [dfa.edges[(state, succ)]['label'] for succ in successors]:
                        divided.add(state)
                if divided:
                    divided_partitions.append(divided)
                    if len(divided) < len(p):
                        divided_partitions.append(list(set(p) - divided))

            if len(divided_partitions) > len(partitions):
                if partition in partitions:
                  partitions.remove(partition)
                partitions.extend(divided_partitions)
                worklist.extend(divided_partitions)

    min_dfa = nx.DiGraph()
    state_mapping = {}  # Mapeo de estados originales a estados minimizados

    for i, partition in enumerate(partitions):
        if partition:
            min_state = ', '.join(sorted(str(state) for state in partition))  # Nuevo nombre de estado como contenido de los estados
            state_mapping.update({state: min_state for state in partition})

    for source, target, label in dfa.edges(data='label'):
        if source in state_mapping:
            min_source = state_mapping[source]
        else:
            print(f"La clave {source} no está presente en state_mapping.")
            # Manejar el error según sea necesario


        min_target = state_mapping.get(target, None)
        if min_target is None:
            print(f"La clave {target} no está presente en state_mapping.")
            # Manejar el error según sea necesario
        else:
          #min_target = state_mapping[target]
          min_dfa.add_edge(min_source, min_target, label=label)

        #min_dfa.add_edge(min_source, min_target, label=label)

    min_dfa.graph['start'] = state_mapping[dfa.graph['start']]
    min_dfa.graph['accept'] = [state_mapping[state] for state in dfa.graph['accept'] if state in state_mapping]

    if '()' in min_dfa.nodes:
        min_dfa.remove_node('()')

        # Asegúrate de también eliminar cualquier arista que apunte a este nodo
        for source, target in list(min_dfa.edges):
            if target == '()':
                min_dfa.remove_edge(source, target)

    return min_dfa

def remove_unreachable_states(dfa):
    # Realiza un análisis de alcanzabilidad desde el estado inicial del DFA
    reachable_states = set()
    stack = [dfa.graph['start']]

    while stack:
        state = stack.pop()
        if state not in reachable_states:
            reachable_states.add(state)
            stack.extend(successor for successor in dfa.successors(state))

    # Elimina los estados inalcanzables y las transiciones asociadas
    unreachable_states = set(dfa.nodes) - reachable_states
    dfa.remove_nodes_from(unreachable_states)

if __name__ == "__main__":
    # AFD
    w = input("Enter a string to check: ")

    remove_unreachable_states(afd)

    # Minimiza el AFD
    VizAFD_Min = hopcroft_minimization(VizAFD, alphabet)

    min_dfa = Simulation_AFDMinimal(afd)


    # Elimina el estado final vacío '()' y sus aristas del AFD minimizado
    if '()' in min_dfa.nodes:
        min_dfa.remove_node('()')

        # Asegúrate de también eliminar cualquier arista que apunte a este nodo
        for source, target in list(min_dfa.edges):
            if target == '()':
                min_dfa.remove_edge(source, target)

    # Dibujar el AFD minimizado
    dot: 'graphviz.graphs.Digraph' = graphviz.Digraph(comment='AFD Minimizado')
    dot.attr(rankdir='LR')
    setStates = set()

    dot.attr(label='AFD Minimizado')

    def draw_state(state: 'State'):
        setStates.add(state.getId())
        dot.node(state.getId(), label=state.value, shape='doublecircle' if state.isFinalState else 'circle')
        for transition in state.transitions:
            for destiny in state.transitions[transition]:
                if destiny.getId() not in setStates:
                    draw_state(destiny)
                dot.edge(state.getId(), destiny.getId(), label=transition)

    draw_state(VizAFD_Min)

    dot.render('AFD_Min'+'.gv', view=True, directory='./AFD_Min')


    symbols = set()
    for _, _, label in min_dfa.edges(data='label'):
            symbols.add(label)

    with open('descripcion_afd_minimizado.txt', 'w') as file:
            file.write("ESTADOS = {}\n".format(sorted(min_dfa.nodes)))
            file.write("SIMBOLOS = {}\n".format(sorted(symbols)))
            file.write("INICIO = {}\n".format(min_dfa.graph['start']))
            file.write("ACEPTACION = {}\n".format(sorted(min_dfa.graph['accept'])))
            file.write("TRANSICIONES = {}\n".format(sorted(min_dfa.edges(data='label'))))

    #SIMULACION DEL AFD
    result = check_membership(min_dfa, w)

    if result:
        print(f"'{w}' belongs to L({regex})")
    else:
        print(f"'{w}' does not belong to L({regex})")


: 

# Construccion Directa

In [ ]:
class Node:
    firstpos = None
    lastpos = None
    nullable = None

    def __init__(self, parent):
        self.parent = parent

    def create_subtree(self, nodestack):
        pass

    def isnullable(self):
        pass

    def findfirstpos(self):
        pass

    def findlastpos(self):
        pass


class ConcatNode(Node):
    def __init__(self, parent):
        super().__init__(parent)
        self.lchild = None
        self.rchild = None
        self.number = 1
        self.string = None

    def create_subtree(self, nodestack):
        print("nodestack: ",nodestack)
        operand2 = nodestack.pop()
        operand1 = nodestack.pop()
        #print("operand2: ",operand2)
        #print("operand1: ", operand1)

        if isinstance(operand1, Node):
            self.lchild = operand1
            operand1.parent = self
        else:
            self.lchild = LeafNode(parent=self, string=operand1)

        if isinstance(operand2, Node):
            self.rchild = operand2
            operand2.parent = self
        else:
            self.rchild = LeafNode(parent=self, string=operand2)

    def __str__(self):
        return '[' + str(self.lchild) + '.' + str(self.rchild) + ']'

    def isnullable(self):
        a = self.lchild.isnullable()
        b = self.rchild.isnullable()
        self.nullable = a and b
        return self.nullable

    def findfirstpos(self):
        a = self.lchild.findfirstpos()
        b = self.rchild.findfirstpos()
        if self.lchild.nullable:
            self.firstpos = list(set(a + b))
        else:
            self.firstpos = a
        return self.firstpos

    def findlastpos(self):
        a=self.lchild.findlastpos()
        b=self.rchild.findlastpos()
        if self.rchild.nullable:
            self.lastpos = list(set( a+ b))
        else:
            self.lastpos = b
        return self.lastpos


class StarNode(Node):
    def __init__(self, parent):
        super().__init__(parent)
        self.child = None
        self.number = 0
        self.string = None

    def create_subtree(self, nodestack):
        operand = nodestack.pop()
        if isinstance(operand, Node):
            self.child = operand
        else:
            self.child = LeafNode(parent=self, string=operand)

    def __str__(self):
        return '[ (' + str(self.child) + ') * ]'

    def isnullable(self):
        self.child.isnullable()
        self.nullable = True
        return True

    def findfirstpos(self):
        self.firstpos = self.child.findfirstpos()
        return self.firstpos

    def findlastpos(self):
        self.lastpos = self.child.findlastpos()
        return self.lastpos


class OrNode(Node):
    def __init__(self, parent):
        super().__init__(parent)
        self.lchild = None
        self.rchild = None
        self.number = 0
        self.string = None

    def create_subtree(self, nodestack):
        operand2 = nodestack.pop()
        operand1 = nodestack.pop()
        if isinstance(operand1, Node):
            self.lchild = operand1
            operand1.parent = self
        else:
            self.lchild = LeafNode(parent=self, string=operand1)

        if isinstance(operand2, Node):
            self.rchild = operand2
            operand2.parent = self
        else:
            self.rchild = LeafNode(parent=self, string=operand2)

    def __str__(self):
        return '[' + str(self.lchild) + '|' + str(self.rchild) + ']'

    def isnullable(self):
        a = self.lchild.isnullable()
        b = self.rchild.isnullable()
        self.nullable = a or b
        return self.nullable

    def findfirstpos(self):
        self.firstpos = list(set(self.lchild.findfirstpos() + self.rchild.findfirstpos()))
        return self.firstpos

    def findlastpos(self):
        self.lastpos = list(set(self.lchild.findlastpos() + self.rchild.findlastpos()))
        return self.lastpos


class LeafNode(Node):
    num_of_instances = 0


    def __init__(self, parent, string):
        super().__init__(parent)
        self.number = 0
        self.string = string

        LeafNode.num_of_instances += 1
        self.number = LeafNode.num_of_instances

    def __str__(self):
        print("self: ",self.string)
        return '[' + self.string + ']'

    def isnullable(self):
        if self.string == 'e':
            self.nullable = True
            return True
        else:
            self.nullable = False
            return False

    def findfirstpos(self):
        if self.string == 'e':
            self.firstpos = []
            return self.firstpos
        else:
            self.firstpos = [self.number, ]
            return self.firstpos

    def findlastpos(self):
        if self.string == 'e':
            self.lastpos = []
            return self.lastpos
        else:
            self.lastpos = [self.number, ]
            return self.lastpos


: 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque

class SyntaxTree:

    def __init__(self, regex_string):
        #print('re = ', self.add_concatenation(regex_string))
        LeafNode.num_of_instances = 0
        self.regex = self.add_concatenation(regex_string) + '.#'
        #print('\nregex = ', self.regex)
        self.root = self.convert_regex_to_syntax_tree()
        #print('tree = ', str(self.root))
        self.root.isnullable()
        self.root.findfirstpos()
        self.root.findlastpos()

        self.follow_pos = [None] * LeafNode.num_of_instances
        self.print_first_pos(self.root)

    def print_first_pos(self, node):
        if isinstance(node, ConcatNode):
            print('Concatenacion', " Firstpos: ", node.firstpos, " Lastpos: ", node.lastpos, " NULLABLE = ", node.nullable)
            self.print_first_pos(node.lchild)
            self.print_first_pos(node.rchild)
        elif isinstance(node, StarNode):
            print('Estrella de Kleene', " Firstpos: ", node.firstpos, " Lastpos: ", node.lastpos, " NULLABLE = ", node.nullable)
            self.print_first_pos(node.child)
        elif isinstance(node, OrNode):
            print('Alternancia (|)', " Firstpos: ", node.firstpos, " Lastpos: ", node.lastpos, " NULLABLE = ", node.nullable)
            self.print_first_pos(node.lchild)
            self.print_first_pos(node.rchild)
        else:
            print(node.number, node.string, " Firstpos: ", node.firstpos, " Lastpos: ", node.lastpos, " NULLABLE = ", node.nullable)

    def add_concatenation(self, string):
      result = ''
      i = 0
      while i < len(string):
          print("len(string): ",len(string))

          if len(string) > 2 and string[-1] != '+':
            if i < len(string) - 1 and string[i + 1] == '+' and string[i+2]!= ')':
                result += '(' + string[i] + '.' + string[i] + '*).'   # Transformar el símbolo '+' en la expresión aa*
                i += 1  # Saltar el símbolo '+'

            elif i < len(string) - 1 and string[i + 1] == '+' and string[i+2]== ')':
              result += string[i] + '.' + string[i] + '*'
              i += 1  # Saltar el símbolo '+'
            else:
              result += string[i]
          elif len(string) > 2 and string[-1] == '+':   #Manejar el signo '+' cuando el ultimo caracter sea +
            if i < len(string) - 1 and string[i + 1] == '+' and i + 2 == len(string):
                print("pasa aca2")
                print("i+2",i+2, " len", len(string))
                result += '(' + string[i] + '.' + string[i] + '*)'   # Transformar el símbolo '+' en la expresión aa*
                i += 1  # Saltar el símbolo '+'

            elif i < len(string) - 1 and string[i + 1] == '+' and i + 2 < len(string):
                print("pasa aca")
                result += '(' + string[i] + '.' + string[i] + '*).'   # Transformar el símbolo '+' en la expresión aa*
                i += 1  # Saltar el símbolo '+'

            elif i < len(string) - 1 and string[i + 1] == '+':
              result += string[i] + '.' + string[i] + '*'
              i += 1  # Saltar el símbolo '+'
            else:
              result += string[i]


          else:
            if i < len(string) - 1 and string[i + 1] == '+':
                result += '(' + string[i] + '.' + string[i] + '*)'   # Transformar el símbolo '+' en la expresión aa*
                i += 1  # Saltar el símbolo '+'
            else:
              result += string[i]


          if i < len(string) - 1 and string[i + 1] == '?' and (i == len(string) - 2 or (i + 2 < len(string) and string[i + 2] != ')' and string[i] != '(')):
            result = result[:-1]
            # Transformar el símbolo '?' en la expresión (a | ε)
            result += '(' + string[i] + '|e'+ ')'
            i += 2  # Saltar el carácter después del '?'
          #result += string[i]

          if i < len(string) - 1 and string[i + 1] == '?':
              #result += string[i]
              # Transformar el símbolo '?' en la expresión a | ε
              result += '|e'
              i += 1  # Saltar el símbolo '?'
          elif (i < len(string) - 1 and string[i].isalnum() and string[i + 1].isalnum()) or \
                  (i < len(string) - 1 and string[i].isalnum() and string[i + 1] == '(') or \
                  (i < len(string) - 1 and string[i] == ')' and string[i + 1].isalnum()) or \
                  (i < len(string) - 1 and string[i] == '*' and string[i + 1].isalnum()) or \
                  (i < len(string) - 1 and string[i] == '*' and string[i + 1] == '(') or \
                  (i < len(string) - 1 and string[i] == ')' and string[i + 1] == '('):
              result += '.'
          i += 1

      print("result: ",result)
      return result


    def not_greater(self, i, j):
        priority = {'*': 3, '.': 2, '|': 1}
        try:
            a = priority[i]
            b = priority[j]
            return True if a <= b else False
        except KeyError:
            return False

    def convert_regex_to_syntax_tree(self):
        node_stack = []
        op_stack = []

        for r in self.regex:
            if r.isalnum() or r == '#':
                node_stack.append(r)
            elif r == '(':
                op_stack.append(r)
            elif r == ')':
                while len(op_stack) != 0 and op_stack[-1] != '(':
                    self.convert_substring_to_subtree(op_stack, node_stack)
                op_stack.pop()  # pop the '('
            else:
                while len(op_stack) != 0 and self.not_greater(r, op_stack[-1]):
                    self.convert_substring_to_subtree(op_stack, node_stack)
                op_stack.append(r)

        while len(op_stack) != 0:
            self.convert_substring_to_subtree(op_stack, node_stack)

        root = node_stack.pop()
        print("root: ", node_stack)
        return root

    def convert_substring_to_subtree(self, op_stack, node_stack):
        op = op_stack.pop()

        if op == '*':
            op = StarNode(parent=None)
        elif op == '.':
            op = ConcatNode(parent=None)
        elif op == '|':
            op = OrNode(parent=None)
        else:
            raise Exception('Unknown Operator!')

        op.create_subtree(node_stack)
        node_stack.append(op)

    def find_follow_pos(self, node):
        if isinstance(node, ConcatNode):
            for i in node.lchild.lastpos:
                if self.follow_pos[i - 1]:
                    self.follow_pos[i - 1] = list(set(self.follow_pos[i - 1] + node.rchild.firstpos))
                else:
                    self.follow_pos[i - 1] = node.rchild.firstpos

            self.find_follow_pos(node.lchild)
            self.find_follow_pos(node.rchild)
        elif isinstance(node, StarNode):
            for i in node.lastpos:
                if self.follow_pos[i - 1]:
                    self.follow_pos[i - 1] = list(set(self.follow_pos[i - 1] + node.firstpos))
                else:
                    self.follow_pos[i - 1] = node.firstpos
            self.find_follow_pos(node.child)


: 

In [ ]:

class State:
    def __init__(self,name,statenumber, accepting = False):
        self.name=name
        self.statenumber=statenumber
        self.accepting = accepting
        self.Dtran={}

    def __str__(self):
        s = f'<{self.name}, {self.statenumber}, '
        for transition, next_state in self.Dtran.items():
            s += f'Dtran({transition})={next_state.name}, '
        s = s.rstrip(', ') + '>\n'
        return s

    def generate_dot_representation(self):
        dot = Digraph(comment=self.name)
        dot.node(str(self.statenumber), label=self.name, shape='circle')
        for transition, next_state in self.Dtran.items():
            dot.edge(str(self.statenumber), str(next_state.statenumber), label=transition)
        return dot


class ConvertToDfa:
    def __init__(self,tree):
        self.tree = tree
        self.followpos = tree.follow_pos
        self.initial_statenumber = tree.root.firstpos
        self.initial_state=None
        self.leaf_nodes={}
        self.seen_states = []

    def find_leaf_nodes(self,node):
        if isinstance(node,LeafNode):
            if node.string!='#' and node.string!='e':
                try:
                    self.leaf_nodes[node.string] += [node.number, ]
                except:
                    self.leaf_nodes[node.string]=[node.number,]
        elif isinstance(node,StarNode):
            self.find_leaf_nodes(node.child)
        else:
            self.find_leaf_nodes(node.lchild)
            self.find_leaf_nodes(node.rchild)

        return

    def convert(self):
        state_dic={1:'A',2:'B',3:'C',4:'D',5:'E',6:'F',7:'G',8:'H',9:'I',10:'J',
                   11:'K',12:'L',13:'M',14:'N',15:'O',16:'P',17:'Q',18:'R',19:'S',20:'T',
                   21:'U',22:'V',23:'W',24:'X',25:'Y',26:'Z'}

        self.find_leaf_nodes(self.tree.root)
        print(self.leaf_nodes)

        self.initial_state=State(name='A',statenumber=self.initial_statenumber)
        x=2

        left_states=[self.initial_state,]
        seen_states=[]
        print('\n')
        while len(left_states)!=0:
            state=left_states.pop()
            if state not in seen_states:
                seen_states.append(state)
                print('-------------------------------------------\n')
                print('estado =', str(state))
                for string in self.leaf_nodes:
                    i=[elem for elem in state.statenumber if elem in self.leaf_nodes[string]]
                    print('caracter leido =',string)
                    if len(i)!=0:
                        next_statenumber = []

                        for item in list(i):
                            next_statenumber= list(set(next_statenumber+ self.followpos[item-1]))

                        for seen in seen_states:
                            if next_statenumber == seen.statenumber:
                                state.Dtran[string] = seen
                                print('pasa al estado: ', str(seen))
                                break
                        else:
                            next_state=State(name=state_dic[x],statenumber=next_statenumber)
                            x=x+1
                            state.Dtran[string] = next_state
                            print('pasa al estado: ', str(next_state))
                            left_states.append(next_state)

        #accepting_states = []
        accepting_state = seen_states[-1]
        #print("Último elemento:", ultimo_elemento)


        #self.accepting_states.append(accepting)

        print('-------------------------------------------')
        return self.initial_state, accepting_state, seen_states

    def write_in_file(self,file):
        left_states=[self.initial_state,]
        seen_states=[]

        while len(left_states)!=0:
            state=left_states.pop()
            if state not in seen_states:
                seen_states.append(state)
                file.write(str(state))
                for transition in state.Dtran:
                    next_state=state.Dtran[transition]
                    left_states.append(next_state)
        file.write('\n\n\n')


: 

In [ ]:
from graphviz import Digraph
import os
import re

class DFA:
    def __init__(self, initial_state, accepting):
        self.current_state = initial_state
        self.accepting_states = accepting
        self.transition_disp = True


    def process_input(self, input_string):
        for char in input_string:
            if char in self.current_state.Dtran:
                next_state = self.current_state.Dtran[char]
                print(f"caracter leido = {char}")
                print(f"pasa al estado: {str(next_state)}")
                self.current_state = next_state
            else:
                print(f"No hay transición para el caracter {char}")
                self.transition_disp = False

    def simulate(self, input_string):
        print("Simulación del AFD:")
        print(f"estado inicial: {str(self.current_state)}")
        self.process_input(input_string)
        #print("self.accepting_states ", self.accepting_states)
        if self.current_state == self.accepting_states and self.transition_disp == True:

          print("Cadena aceptada.")
        else:
          print("Cadena no aceptada.")

        print("Simulación completada.")

    def parse_state_string(state_string):
    # Utilizamos expresiones regulares para extraer información
      match = re.match(r'<([A-Z]), \[([0-9, ]+)\], Dtran\((\w+)\)=([A-Z]), Dtran\((\w+)\)=([A-Z])>', state_string)
      if match:
          name = match.group(1)
          symbols = [int(x) for x in match.group(2).split(',')]
          transitions = {match.group(3): match.group(4), match.group(5): match.group(6)}
          return State(name, symbols, transitions)
      else:
          raise ValueError("Formato de estado no válido")

    def generate_dfa_construction(self, seen_states, image_folder):
        if not os.path.exists(image_folder):
            os.makedirs(image_folder)

        dot = Digraph(comment='DFA Construction')
        dot.graph_attr['rankdir'] = 'LR'

        for state in seen_states:
            dot.node(str(state.statenumber), label=str(state), shape='circle')
            for transition, next_state in state.Dtran.items():
                dot.edge(str(state.statenumber), str(next_state.statenumber), label=f"{transition}")

        image_filename = os.path.join(image_folder, f'DFA_construction')
        dot.render(image_filename, format='png', cleanup=True)


: 

In [ ]:
file = open('load.txt', 'r')

inp = file.read()
inp = inp.split('\n')

print(inp)

converttree = None

image_folder = "dfa_images"

for caracter in inp:
    regex = caracter
    tree = SyntaxTree(regex)
    tree.find_follow_pos(tree.root)
    print(tree.follow_pos)



    #tree.visualize_syntax_tree()

    # Crear el DFA y simular la entrada
    converttree=ConvertToDfa(tree=tree)
    dfa, accepting_states, seen_states = converttree.convert()
    print("full:_states",accepting_states)
    dfa_simulate = DFA(dfa, accepting_states)
    input_string = input("Ingrese la cadena a evaluar: ")
    dfa_simulate.simulate(input_string)

    if converttree is None:
        converttree = ConvertToDfa(tree=tree)
    else:
        converttree.tree = tree

    #tree.visualize_syntax_tree()

    # Agregar la opción para visualizar el árbol de DFA
    choice = input("¿Desea visualizar la construcción de DFA? (S/N): ")
    if choice.upper() == "S":
        image_folder = "dfa_construction_images"
        dfa_simulate.generate_dfa_construction(seen_states, image_folder)
    else:
        print("Ok")


    outputfile=open('output.txt','a')
    #converttree.write_in_file(outputfile)

: 

In [1]:
import re

In [4]:
class YALexParser:
    def __init__(self, content):
        self.content = self._remove_comments(content)
        self.regular_definitions = {}
        self.rules = []
        self.header = ""
        self.trailer = ""

    def _remove_comments(self, content):
        return re.sub(r'(\(\*.*?\*\))', '', content, flags=re.DOTALL)

    def parse(self):
        self._parse_header_trailer()
        self._parse_regular_definitions()
        self._parse_rules()

    def _parse_header_trailer(self):
        header_pattern = re.compile(r'\{\s*header\s*\}\s*(\{[^\}]*\})', re.DOTALL)
        trailer_pattern = re.compile(r'\{\s*trailer\s*\}\s*(\{[^\}]*\})', re.DOTALL)
        
        header_match = header_pattern.search(self.content)
        if header_match:
            self.header = header_match.group(1)
        
        trailer_match = trailer_pattern.search(self.content)
        if trailer_match:
            self.trailer = trailer_match.group(1)

    def _parse_regular_definitions(self):
        regex_def_pattern = re.compile(r'let\s+(\w+)\s*=\s*([^#]*?)\n(?=let|$)', re.DOTALL)
        for match in regex_def_pattern.finditer(self.content):
            ident, regexp = match.groups()
            self.regular_definitions[ident] = regexp.strip()

    def _parse_rules(self):
        rule_pattern = re.compile(r'rule\s+(\w+)\s*=\s*(.*?)\s*\{([^}]*)\}', re.DOTALL)
        for match in rule_pattern.finditer(self.content):
            entrypoint, conditions, action = match.groups()
            self.rules.append((entrypoint, conditions.strip(), action.strip()))

    def get_regular_definitions(self):
        return self.regular_definitions

    def get_rules(self):
        return self.rules

    def get_header(self):
        return self.header

    def get_trailer(self):
        return self.trailer

In [8]:
file_content = open('slr-1.yal', 'r')
inp = file_content.read()

parser = YALexParser(inp)
parser.parse()


print("Header:", parser.get_header())
print("Regular Definitions:", parser.get_regular_definitions())
print("Rules:", parser.get_rules())
print("Trailer:", parser.get_trailer())

Header: 
Regular Definitions: {'delim': "[' ''\\t''\\n']", 'ws': 'delim+', 'letter': "['A'-'Z''a'-'z']", 'digit': "['0'-'9']", 'id': "letter(letter|digit)*\n\nrule tokens =\n    ws\n  | id        { return ID }               \n  | '+'       { return PLUS }\n  | '*'       { return TIMES }\n  | '('       { return LPAREN }\n  | ')'       { return RPAREN }"}
Rules: [('tokens', 'ws\n  | id', 'return ID')]
Trailer: 
